# **Crime Analytics - Visualising Incident Reports**

In this notebook, we will be analysing the San Francisco criminal incident data as contained in the police department incident reports published on San Francisco's open data web portal: [https://data.sfgov.org](https://data.sfgov.org/Public-Safety/Police-Department-Incident-Reports-2018-to-Present/wg3w-h783). Note that whilst the original instructions were to use incident data from the summer of 2014, this is now 13 years ago so the underlying dataset has been updated to a full year of data from the summer of 2022.

The raw data contained 136,677 rows of data across 35 columns:

| Field Name | Data Type |
| ----------- | --------- |
| Incident Datetime | Object |
| Incident Date | Object |
| Incident Time | Object |
| Incident Year | Integer |
| Incident Day of Week | Object |
| Report Datetime | Object |
| Row ID | Float |
| Incident ID | Integer |
| Incident Number | Integer |
| CAD Number | Object |
| Report Type Code | Object |
| Report Type Description | Object |
| Filed Online | Object |
| Incident Code | Integer |
| Incident Category | Object |
| Incident Subcategory | Object |
| Incident Description | Object |
| Resolution | Object |
| Intersection | Object |
| CNN | Float |
| Police District | Object |
| Analysis Neighborhood | Object |
| Supervisor District | Float |
| Supervisor District 2012 | Float |
| Latitude | Float |
| Longitude | Float |
| Point | Object |
| Neighborhoods | Float |
| ESNCAG - Boundary File | Float |
| Central Market/Tenderloin Boundary Polygon - Updated | Float |
| Civic Center Harm Reduction Project Boundary | Float |
| HSOC Zones as of 2018-06-05 | Float |
| Invest In Neighborhoods (IIN) Areas | Float |
| Current Supervisor Districts | Float |
| Current Police Districts | Float |

For field descriptions see the [Field Definitions](https://datasf.gitbook.io/datasf-dataset-explainers/sfpd-incident-report-2018-to-present#field-definitions).

In [1]:
# import all necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import seaborn as sns

In [2]:
import plotly.io as pio
# pio.renderers
pio.renderers.default = "iframe"

In [3]:
incidents = pd.read_csv('sf_incidents_2223.csv')
# clean up the column names
incidents.columns = incidents.columns.str.lower().str.replace(' ', '_')
# select the 'useful' columns for the analysis
useful_cols = [col for col in incidents if col.startswith('incident') or col in ['resolution', 'police_district', 'analysis_neighborhood', 'supervisor_district', 'neighborhoods',
                                                                                 'latitude', 'longitude', 'point']]
incidents = incidents.loc[:, useful_cols]
# convert incident_datetime to a datetime column
incidents['incident_datetime'] = pd.to_datetime(incidents.incident_datetime, infer_datetime_format = True)
incidents['incident_date'] = pd.to_datetime(incidents.incident_date, infer_datetime_format = True)
incidents.info()

# add hour and month information
incidents['incident_hour'] = incidents.incident_datetime.dt.hour
incidents['incident_month_num'] = incidents.incident_datetime.dt.month
incidents['incident_month'] = incidents.incident_datetime.dt.month_name()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136677 entries, 0 to 136676
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   incident_datetime      136677 non-null  datetime64[ns]
 1   incident_date          136677 non-null  datetime64[ns]
 2   incident_time          136677 non-null  object        
 3   incident_year          136677 non-null  int64         
 4   incident_day_of_week   136677 non-null  object        
 5   incident_id            136677 non-null  int64         
 6   incident_number        136677 non-null  int64         
 7   incident_code          136677 non-null  int64         
 8   incident_category      136545 non-null  object        
 9   incident_subcategory   136545 non-null  object        
 10  incident_description   136677 non-null  object        
 11  resolution             136677 non-null  object        
 12  police_district        136677 non-null  obje

In [4]:
incidents.iloc[:5, :12]

,incident_datetime,incident_date,incident_time,incident_year,incident_day_of_week,incident_id,incident_number,incident_code,incident_category,incident_subcategory,incident_description,resolution
0,2023-05-31 23:49:00,2023-05-31,23:49,2023,Wednesday,1281250,230377142,19057,Disorderly Conduct,Intimidation,Terrorist Threats,Open or Active
1,2023-05-31 23:45:00,2023-05-31,23:45,2023,Wednesday,1281464,230377948,75000,Missing Person,Missing Person,Found Person,Open or Active
2,2023-05-31 23:45:00,2023-05-31,23:45,2023,Wednesday,1281464,230377948,74000,Missing Person,Missing Adult,Missing Adult,Open or Active
3,2023-05-31 23:44:00,2023-05-31,23:44,2023,Wednesday,1281275,230377136,16210,Drug Offense,Drug Violation,Opiates Offense,Cite or Arrest Adult
4,2023-05-31 23:44:00,2023-05-31,23:44,2023,Wednesday,1281275,230377136,62050,Warrant,Warrant,"Warrant Arrest, Enroute To Outside Jurisdiction",Cite or Arrest Adult


In [5]:
incidents.iloc[:5, 12:]

,police_district,analysis_neighborhood,supervisor_district,latitude,longitude,point,neighborhoods,incident_hour,incident_month_num,incident_month
0,Tenderloin,Tenderloin,5.0,37.785790,-122.412970,POINT (-122.41296966814406 37.78578958358186),20.0,23,5,May
1,Taraval,Lakeshore,7.0,37.719056,-122.481424,POINT (-122.48142396348878 37.71905643663192),42.0,23,5,May
2,Taraval,Lakeshore,7.0,37.719056,-122.481424,POINT (-122.48142396348878 37.71905643663192),42.0,23,5,May
3,Northern,Tenderloin,5.0,37.783101,-122.419182,POINT (-122.41918170505187 37.78310139923345),20.0,23,5,May
4,Northern,Tenderloin,5.0,37.783101,-122.419182,POINT (-122.41918170505187 37.78310139923345),20.0,23,5,May


There are multiple records for each unit incident id/number. The plot below clearly indicates that most incidents only have a single record attached to them, but about 10% (16,617) have up to 4 records. The documentation indicates that a single incident report can have one or more incident codes attached; for example if the police officer discovers narcotics whilst making an arrest (for a non-narcotics related issue), then both an arrest and narcotics incident code would be reported against the same report id. Unless otherwise specified, the analysis will be at the incident code, rather than report level.

In [6]:
counts = pd.DataFrame(incidents.incident_id.value_counts()).reset_index()
counts = counts.groupby("incident_id").count().reset_index()
counts['prop'] = counts['index'] / incidents.shape[0] * 100
counts.columns = ["id_count", "count", "prop"]

fig = px.bar(data_frame=counts, x="id_count", y="prop",
             hover_data={'prop':':.2f',
                         'count':':,'},
             labels={'prop':'Percentage (%)', 'id_count':'Occurrence'},
             title='Proportion of records by incident id occurrence',
             template='simple_white')
fig.update_layout(width=700, height=500) 
fig.show(renderer='iframe')

In [7]:
# clean up the incident categories
incidents['incident_category_new'] = np.where(incidents.incident_category == 'Other Miscellaneous', 'Other',
                                     np.where(incidents.incident_category == 'Motor Vehicle Theft?', 'Motor Vehicle Theft',
                                     np.where(incidents.incident_category == 'Human Trafficking (A), Commercial Sex Acts', 'Human Trafficking, Commercial Sex Acts',
                                     np.where(incidents.incident_category == 'Weapons Offence', 'Weapons Offense',
                                     np.where(incidents.incident_category == 'Suspicious', 'Suspicious Occ',
                                     incidents.incident_category)))))

The plot below indicates that the most common incident reported in the twelve months between 1 June 2022 and 31 May 2023 was larceny/theft (30.9%). The secondmost common incident category reported was 'Other' (6.9%), and malicious mischief (6.7%), assault (6.4%) and motor vehicle theft (6.3%) rounded out the top 5.

At the opposite end of the spectrum, the least common incidents included civil sidewalks, gambling, human trafficking/commercial sex acts, rape and homicide.

In [8]:
by_cat = pd.DataFrame(incidents.incident_category_new.value_counts())
by_cat.reset_index(inplace=True)
by_cat.columns = ['category', 'count']
by_cat['prop'] = by_cat['count']/incidents.shape[0]*100
by_cat.sort_values(by='prop', inplace=True)

fig = px.bar(data_frame=by_cat[-20:], x='prop', y='category',
             hover_data={'prop':':.2f',
                         'count':':,'},
             labels={'prop':'Percentage (%)', 'category':'Category'},
             title='Proportion of top 20 incidents by category',
             template='simple_white')
fig.update_layout(width=1000) 
fig

This next plot indicates that there are no drastic changes in the profile of incident categories over time. However, it is interesting to note that non-motor vehicle larceny/theft is at its lowest in May and gradually increases to a peak in December. In contrast, drug offenses are highest in July, August and February.

In [9]:
# let's see if there is any pattern in the number of incidents by month of the year
by_month = pd.DataFrame(incidents.groupby(['incident_month', 'incident_month_num', 'incident_category_new'])['incident_id'].count())
by_month.reset_index(inplace=True)
by_month.columns = ['month', 'month_num', 'category', 'count']
by_month['month_sum'] = by_month.groupby('month')['count'].transform('sum')
by_month['total_prop'] = by_month['count']/incidents.shape[0]*100
by_month['month_prop'] = by_month['count']/by_month.month_sum*100

plot_data = by_month.sort_values(by=['month_num', 'month_prop'], ascending=[True, False])
fig = px.bar(plot_data, x='month', y='month_prop', color='category',
             hover_data={'month_prop':':.2f',
                         'count':':,'},
             labels={'month':'Month', 'month_prop':'Percentage (%)', 'category':'Category'},
             title='Proportion of incidents by category and calendar month',
             template='simple_white')
fig.update_layout(height=700)
fig

Now, the following two plots are far more interesting as they indicates some clear patterns regarding when certain types of incidents are most likely to occur. For example, larceny/theft incidents spiked in the late morning from about 10am and again in the early evening/evening between 6pm and 8pm. Similarly:

- burgleries typically occurred at night or in the early morning between 3am and 6am
- vandalism (malicious mischief) typically occurred when it was darker between about 8pm and 6am the following morning
- motor vehicle thefts were more prevalent in the evening from about 5pm onwards until midnight
- fraud incidents were most likely to occur just after midnight
- robberies between 4pm in the afternoon and 6am the following morning, and
- drug offenses overwhelmingly in the daylight hours between 7am and 7pm.

In [10]:
# add parts of the day
incidents['incident_part_of_day'] = np.where(incidents.incident_hour < 5, 'night',
                                    np.where((incidents.incident_hour >= 5) & (incidents.incident_hour < 8), 'early morning',
                                    np.where((incidents.incident_hour >= 8) & (incidents.incident_hour < 11), 'morning',
                                    np.where((incidents.incident_hour >= 11) & (incidents.incident_hour < 12), 'late morning',
                                    np.where((incidents.incident_hour >= 12) & (incidents.incident_hour < 15), 'early afternoon',
                                    np.where((incidents.incident_hour >= 15) & (incidents.incident_hour < 16), 'afternoon',
                                    np.where((incidents.incident_hour >= 16) & (incidents.incident_hour < 17), 'late afternoon',
                                    np.where((incidents.incident_hour >= 17) & (incidents.incident_hour < 19), 'early evening',
                                    np.where((incidents.incident_hour >= 19) & (incidents.incident_hour < 21), 'evening',
                                    np.where(incidents.incident_hour >= 21, 'night', 'TBC'))))))))))

part_of_day_dict = {'early morning': 1,
                 'morning': 2,
                 'late morning': 3,
                 'early afternoon': 4,
                 'afternoon': 5,
                 'late afternoon': 6,
                 'early evening': 7,
                 'evening': 8,
                 'night': 9}
incidents['incident_part_of_day_num'] = incidents.incident_part_of_day.map(part_of_day_dict)

by_part_of_day = pd.DataFrame(incidents.groupby(['incident_part_of_day', 'incident_part_of_day_num', 'incident_category_new'])['incident_id'].count())
by_part_of_day.reset_index(inplace=True)
by_part_of_day.columns = ['part_of_day', 'part_of_day_num', 'category', 'count']
by_part_of_day['day_sum'] = by_part_of_day.groupby('part_of_day')['count'].transform('sum')
by_part_of_day['total_prop'] = by_part_of_day['count']/incidents.shape[0]*100
by_part_of_day['part_prop'] = by_part_of_day['count']/by_part_of_day.day_sum*100

plot_data = by_part_of_day.sort_values(by=['part_of_day_num', 'part_prop'], ascending=[True, False])
fig = px.bar(plot_data, x='part_of_day', y='part_prop', color='category',
             hover_data={'part_prop':':.2f',
                         'count':':,'},
             labels={'part_of_day':'Hour', 'part_prop':'Percentage (%)', 'category':'Category'},
             title='Proportion of incidents by category and part of the day',
             template='simple_white')
fig.update_layout(height=700)
fig

In [11]:
by_hour = pd.DataFrame(incidents.groupby(['incident_hour', 'incident_category_new'])['incident_id'].count())
by_hour.reset_index(inplace=True)
by_hour.columns = ['hour', 'category', 'count']
by_hour['day_sum'] = by_hour.groupby('hour')['count'].transform('sum')
by_hour['total_prop'] = by_hour['count']/incidents.shape[0]*100
by_hour['hour_prop'] = by_hour['count']/by_hour.day_sum*100

plot_data = by_hour.sort_values(by=['hour', 'hour_prop'], ascending=[True, False])
fig = px.bar(plot_data, x='hour', y='hour_prop', color='category',
             hover_data={'hour_prop':':.2f',
                         'count':':,'},
             labels={'hour':'Hour', 'hour_prop':'Percentage (%)', 'category':'Category'},
             title='Proportion of incidents by category and hour',
             template='simple_white')
fig.update_layout(height=700)
fig

In [12]:
by_police_district = pd.DataFrame(incidents.groupby(['police_district', 'incident_category_new'])['incident_id'].count())
by_police_district.reset_index(inplace=True)
by_police_district.columns = ['district', 'category', 'count']
by_police_district['district_sum'] = by_police_district.groupby('district')['count'].transform('sum')
by_police_district['district_prop'] = by_police_district['count']/by_police_district.district_sum*100

plot_data = by_police_district.sort_values(by=['district', 'district_prop'], ascending=[True, False])
fig = px.bar(plot_data, x='district', y='district_prop', color='category',
             hover_data={'district_prop':':.2f',
                         'count':':,'},
             labels={'district':'Police District', 'district_prop':'Percentage (%)', 'category':'Category'},
             title='Proportion of incidents by category and police district',
             template='simple_white')
fig.update_layout(height=700)
fig

In [13]:
by_supervisor_district = pd.DataFrame(incidents.groupby(['supervisor_district', 'incident_category_new'])['incident_id'].count())
by_supervisor_district.reset_index(inplace=True)
by_supervisor_district.columns = ['district', 'category', 'count']
by_supervisor_district['district_sum'] = by_supervisor_district.groupby('district')['count'].transform('sum')
by_supervisor_district['district_prop'] = by_supervisor_district['count']/by_supervisor_district.district_sum*100

plot_data = by_supervisor_district.sort_values(by=['district', 'district_prop'], ascending=[True, False])
fig = px.bar(plot_data, x='district', y='district_prop', color='category',
             hover_data={'district_prop':':.2f',
                         'count':':,'},
             labels={'district':'Supervisor District', 'district_prop':'Percentage (%)', 'category':'Category'},
             title='Proportion of incidents by category and supervisor district',
             template='simple_white')
fig.update_layout(height=700)
fig

In [14]:
by_neighbourhood = pd.DataFrame(incidents.groupby(['analysis_neighborhood', 'neighborhoods', 'incident_category_new'])['incident_id'].count())
by_neighbourhood.reset_index(inplace=True)
by_neighbourhood.columns = ['name', 'id', 'category', 'count']
by_neighbourhood['neighbourhood_sum'] = by_neighbourhood.groupby('name')['count'].transform('sum')
by_neighbourhood['neighbourhood_prop'] = by_neighbourhood['count']/by_neighbourhood.neighbourhood_sum*100

plot_data = by_neighbourhood.sort_values(by=['name', 'neighbourhood_prop'], ascending=[True, False])
fig = px.bar(plot_data, x='name', y='neighbourhood_prop', color='category',
             hover_data={'neighbourhood_prop':':.2f',
                         'count':':,'},
             labels={'name':'Neighbourhood', 'neighbourhood_prop':'Percentage (%)', 'category':'Category'},
             title='Proportion of incidents by category and neighbourhood',
             template='simple_white')
fig.update_layout(height=700)
fig